<a href="https://colab.research.google.com/github/fambargh/SAMPLE/blob/master/unet_seg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
pip install keras-segmentation


  Created wheel for keras-segmentation: filename=keras_segmentation-0.2.0-cp36-none-any.whl size=25694 sha256=20adf91977917b0a396e4005aa5491be3fcb1930ae538f3706ecf88dcdf713a2
  Stored in directory: /root/.cache/pip/wheels/10/43/34/8d5a4aaa8d4479380c3aef57abf90ac47aa1e062c772defa39
Successfully built keras-segmentation


In [8]:
!python -m keras_segmentation verify_dataset \
 --images_path="/content/image_train/" \
 --segs_path="/content/annotate_train/"  \
 --n_classes=256

Using TensorFlow backend.
100% 48/48 [00:00<00:00, 1204.35it/s]
Dataset verified! 
verify_dataset


In [10]:
!python -m keras_segmentation visualize_dataset \
 --images_path="dataset_path/images_prepped_train/" \
 --segs_path="dataset_path/annotations_prepped_train/"  \
 --n_classes=256

Using TensorFlow backend.
Press any key to navigate. 
visualize_dataset


In [11]:
import keras_segmentation

model = keras_segmentation.models.unet.vgg_unet(n_classes=256 ,  input_height=128, input_width=128  )

Using TensorFlow backend.






58892288/58889256 [==============================] - 6s 0us/step











In [37]:
model.train( 
    train_images =  "/content/image_train/",
    train_annotations = "/content/annotate_train/",
    checkpoints_path = "/content/checkpoint/vgg_unet_1" , epochs=5
)

100%|██████████| 48/48 [00:00<00:00, 1172.32it/s]

Verifying train dataset
Dataset verified! 
Starting Epoch  0


Epoch 1/1
512/512 [==============================] - 70s 137ms/step - loss: 0.0041 - acc: 0.9983
saved  /content/checkpoint/vgg_unet_1.model.0
Finished Epoch 0
Starting Epoch  1
Epoch 1/1
512/512 [==============================] - 70s 136ms/step - loss: 0.0033 - acc: 0.9986
saved  /content/checkpoint/vgg_unet_1.model.1
Finished Epoch 1
Starting Epoch  2
Epoch 1/1


RuntimeError: ignored

In [0]:
out = model.predict_segmentation(
    inp="/content/image_test/N-1-L.png",
    out_fname="output.png"
)

In [4]:
from zipfile import ZipFile
filename = "annotate_train.zip"

with ZipFile(filename,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [15]:
from zipfile import ZipFile
filename = "image_test.zip"

with ZipFile(filename,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [18]:
pip install keras-segmentation


In [0]:
from keras import Input
from keras import layers
#from keras import Dropout
from keras.layers import UpSampling2D

In [0]:
img_input = Input(shape=(128,128 , 3 ))

conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(img_input)
conv1 = layers.Dropout(0.2)(conv1)
conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = layers.MaxPooling2D((2, 2))(conv1)

conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = layers.Dropout(0.2)(conv2)
conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = layers.MaxPooling2D((2, 2))(conv2)
conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = layers.Dropout(0.2)(conv3)
conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

up1 = layers.concatenate([UpSampling2D((2, 2))(conv3), conv2], axis=-1)
conv4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
conv4 = layers.Dropout(0.2)(conv4)
conv4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)

up2 = layers.concatenate([UpSampling2D((2, 2))(conv4), conv1], axis=-1)
conv5 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
conv5 = layers.Dropout(0.2)(conv5)
conv5 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)
out = layers.Conv2D( 256, (1, 1) , padding='same')(conv5)

from keras_segmentation.models.model_utils import get_segmentation_model

model = get_segmentation_model(img_input ,  out ) # this would build the segmentation model

In [36]:
model.train( 
    train_images =  "/content/image_train/",
    train_annotations = "/content/annotate_train/",
    checkpoints_path = "/content/checkpoint/vgg_unet_1" , epochs=5
)

100%|██████████| 48/48 [00:00<00:00, 1306.11it/s]

Verifying train dataset
Dataset verified! 
Starting Epoch  0


Epoch 1/1
512/512 [==============================] - 68s 133ms/step - loss: 0.0085 - acc: 0.9966
saved  /content/checkpoint/vgg_unet_1.model.0
Finished Epoch 0
Starting Epoch  1
Epoch 1/1
512/512 [==============================] - 67s 131ms/step - loss: 0.0051 - acc: 0.9978
saved  /content/checkpoint/vgg_unet_1.model.1
Finished Epoch 1
Starting Epoch  2
Epoch 1/1


RuntimeError: ignored

In [0]:
out = model.predict_segmentation(
    inp="/content/image_test/N-1-L.png",
    out_fname="output1.png"
)